In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# ======================
# Load CIFAR10
# ======================

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# ======================
# Data augmentation
# ======================

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomRotation(0.1),
])

# ======================
# CNN migliore (VGG style)
# ======================

def conv_block(filters):
    return [
        layers.Conv2D(filters, 3, padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(filters, 3, padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D()
    ]

model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,

    *conv_block(64),
    *conv_block(128),
    *conv_block(256),

    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")
])

# ======================
# Optimizer (meglio di Adam)
# ======================

optimizer = keras.optimizers.SGD(
    learning_rate=0.1,
    momentum=0.9,
    weight_decay=5e-4
)

model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# ======================
# Scheduler (importantissimo)
# ======================

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 0.1 * (0.1 ** (epoch // 60))
)

# ======================
# Training
# ======================

history = model.fit(
    x_train, y_train,
    epochs=150,
    batch_size=128,
    validation_split=0.1,
    callbacks=[lr_schedule]
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)
